# EE 578B Convex Optimization Project
## Kyle Hadley

## Introduction

Working within the aviation industry, one of the most interesting convex optimization problems that airlines face involves deciding which routes should be offered to their customers. There are many different factors that are involved in choosing the routes: passenger demand, cost of operating route, aircraft available to operate the route, etc.

Based on these factors, there have been predominately two approaches to planning airline's routes: (1) the hub-and-spoke model or (2) point-to-point.

![point_hub_network](point_hub_network.png)

The hub-and-spoke model mimics the spokes of a wheel spreading out from a single; typically airlines has a few select hubs (depending on an airline's size) that all flights fly through. An example of this is Alaska Airlines hub based out of the Seattle-Tacoma airport. The majority of the flights operated by Alaska Air flows through this airport. In this model, the airline chooses to connect as many cities to a single hub such that customers can access a larger network of cities, but are likely to require at least 1 or 2 layover flights. This enables an airline to operate larger aircraft between major hubs (e.g. Los Angeles and New York) and then utilize smaller, regional aircraft for any small flights from the hub to smaller cities (e.g. Log Angeles to San Diego).

The point-to-point model instead attempts to connect as many cities directly rather than through a single hub. The point-to-point model emphasizes flying between two cities directly, regardless of size. An example of this approach is RyanAir which operates many low-cost flights through Europe continent. RyanAir attempts to connect as many cities, regardless of the size of cities to provide easy to access flights. Airlines can often more easily ensure that their routes are meeting the customer demand for a specific pair of cities as they don't need to be as concerned with ensuring "connecting" flights are supported.

There are certainly disadvantages to both solutions. A hub-and-spoke model often leads to higher facility costs and requires passengers to often make connecting flights to reach their final destination (ultimately leading to longer travel times). A point-to-point model also requires significantly more aircraft in a given fleet to support the same number of routes; sometimes this may mean that cities cannot be supported within a point-to-point model if an airline does not have adequate aircraft within their fleet.

Through my project, I hope to provide the foundation of a tool that could allow airlines to more quickly plan their routes based on customer demand. The model will compare multiple networks connected cities - following the hub-and-spoke model, point-to-point, and a combination of the two - to identify the optimal routes to maximize customer reward.

## Sample Airline for Analysis

To help develop the preliminary aspects of the model, I'll be imagining that my model is to be used for an upstart airline - *Concave Airlines* - to help them identify how they should offer routes between various cities to support customer demands. *Concave Airlines* is based out of Las Vegas, Nevada and is focused on mainly offering routes along the west coast and mountain states of the US, but also is interested in potentially supporting other major cities across the US and in North America.

The following cities are currently of interest to be serviced by the airline:
- Las Vegas, NV
- Reno, NV
- Salt Lake City, UT
- Seattle, WA
- Los Angeles, CA
- San Francisco, CA

![graph of cities](City_Connections.png)

# Problem Setup

## Initial Model

We can model our routes as a graph and calculate the minimization akin to the approach taken when modeling shortest path problems with edges formulated. Thus, we can think of the model generically as minimizing the cost of mass flow such that:

$$\min\limits_x c^Tx$$
$$\text{s.t.} \; Ex = b, x \geq 0$$

Within our example, the nodes of our graph represents the cities our airline will service and each edge represents a flight route that our airline will support between two pair cities.

## Graph Structures

In order to leverage this general linear program, we must must have a defined graph that can be used to define the various variables and constraints in the model. However, as we discussed before, there many combinations of flight routes that *Concave Airlines* could service. It may be advantageous to follow a hub-and-spoke model, requiring all flights to flow through Las Vegas (the airlines main hub). Or the airline could choose to connect all the cities via a point-to-point model.

Thus, we can create two graphs to compare the cost of utilizing the two methods we've already discussed. Once we've created each graph, we can identify which graph yields the lowest travel cost when servicing the same city pairs (e.g. if we have $X$ flights between city A and B, which would yield the lowest travel cost). Travel cost in our case is measured as flight minutes or hours.

For both of our graphs that we will be evaluating, the node numbering maps to the following city names:

1. Las Vegas
2. Salt Lake City
3. Los Angeles
4. San Franscisco
5. Reno
6. Seattle

Our first graph will utilize the hub-and-spoke model as discussed above; the graph is illustrated below:

![hub_and_spoke model](Hub_and_spoke_model.png)

From this graph, we can see that we have 6 nodes (i.e. cities) and 12 total edges (i.e. flight routes). Thus, we can see that our $E \in \mathbb{R}^{6x12}$ and $b \in \mathbb{R}^{6}$.

Our second graph will utilize more of a point-to-point model as discussed above; the graph is illustrated below:

![point-to-point model](point_to_point_model.png)

From this graph, we can see that we have the same 6 nodes but 16 total edges. Thus, we can see that our $E \in \mathbb{R}^{6x16}$ and $b \in \mathbb{R}^{6}$.

As discussed in the next section, our $c$ matrix represents the cost of traveling along a given edge - we can estimate this using Google Maps estimates of travel time by flight between the city pairs.

## Model Modifications

Once of the initial issues that can be seen with the initial linear program is that the initial program assumes that we are only moving mass (i.e. our customers) from an single initial node to a single final node. However, *Concave Airlines* will not just be servicing a single set of passengers between two cities, there will be numerous flights between these various cities on a given day. Thus, in order to account for numerous flights, we need our model to account for having multiple mass flows, initial, and final nodes.

We can update our model to account for this by modifying our equation such that,

$$\min\limits_{x} \sum c^Tx_i$$
$$\text{s.t.} \; Ex_i = b_i, x \geq 0 \forall i$$

where $x_i$ represents a single flight between two cities as designated by our matrix $b_i$. We can maintain our minimization and inequality term as we want to minimize all of $x$ and we still want $x$ to be greater than or equal to zero.

For the purposes of this initial sample code, a random set of $100$ routes will be generated between the cities in our graph. The randomization of routes generated will be a function of population of the departure city - as it's reasonable to assume more people are leaving a city with a higher population than a smaller one. We will then compare the resulting travel cost for these $100$ routes against the two graphs discussed above.

The last modification that will be made to our model is the incorporation of costs associated with edges themselves. As normally experienced by passengers, there is a travel cost associated with the airport itself when leaving an airport - both when making a connecting flight or on the first flight of the journey. In order to account for this, we can incorporate a cost $d$ at each edge in $E_o$ such that our cost can be written as: $(c^T + d^TE_o)$. Thus, we re-write our problem as,

$$\min\limits_{x} \sum (c^T + d^TE_o)x_i$$
$$\text{s.t.} \; Ex_i = b_i, x \geq 0 \forall i$$

Solving for this minimization problem should calculate the total flight hours required to support the $100$ flights of interest.

# Optimization Problems

## Primal Problem

In the previous section, we defined the primal problem already; thus it can be written as,

$$\min\limits_{x} \sum\limits_{i=1}^{n} (c^T + d^TE_o)x_i$$
$$\text{s.t.} \; Ex_i = b_i, x \geq 0 \forall i$$

In our primal problem, our variables represent the following:
- $c^T$ represents the travel cost along a given edge - i.e. the time to flight from one city to another, in minutes
- $d^T$ represents the cost of traveling through a city, in minutes; this will be calculated as function of the population of each city
- $E$ represents the graph flow which is solved as $E_i - E_o$
- $x$ represents the flow of people / flights through our graph
- $i$ represents a specific flight; thus, we attempting to calculate the cost of all flights through our graph
- $b_i$ represents the departure and arrival cities for our given flight $i$
- $n$ is the number of flights we'll be assessing

The objective is ultimately attempting to calculate the total flight hours of the fleet of aircraft for the $100$ flights randomly generated. The first constraint - $Ex_i = b_i$ - ensures that a flight moves from its arrival to its destination along viable flight paths (as designated by our graph). The second constraint - $x >= 0$ - ensures that flights don't move backwards through our graph structure (which makes sense, once we depart from a city we don't want to return to it to reach out destination).

## Dual problem

Solving for the dual optimization problem, we can solve for the Lagrangian of our problem as follows,

$$\mathcal{L}(x,v,\mu) = \sum\limits_{i=1}^{n}(c^T+d^TE_o)x_i + \sum\limits_{i=1}^{n} v^T(Ex_i - b_i) - \sum\limits_{i=1}^{n}\mu^T x_i$$

We recall from our previous homeworks and lecture notes that we can solve for our dual constraint by taking the derivative of this function and setting it equal to zero; thus,

$$\frac{\partial}{\partial x}\mathcal{L}(x,v,\mu) = (c^T+d^TE_o) + v^TE - \mu^T = 0$$

Thus, our dual constraint is $(c^T+d^TE_o) + v^TE - \mu^T = 0$. Plugging in our dual constraint into our Lagrangian, we can see that our dual objective is $\sum\limits_{i=1}^{100}v^Tb_i$. Thus, we can re-write our dual problem as,

$$\max\limits_{v, \mu} \sum\limits_{i=1}^{n} v^Tb_i$$
$$\text{s.t.} \; (c^T+d^TE_o) + v^TE - \mu^T = 0, \mu \geq 0$$

In our dual problem, our variables represent the following:
- $v$ represents our value function, which the total cost to go from our initial state to our final state - i.e. the cost from destination to arrival
- $\mu$ represents ineffiences along each edge, which should become zero as we solve for our optimal value


# Applications

Both the primal and dual optimization problem are solved in the code included at the end of this PDF. As mentioned previously, our nodes consist of the cities we wish to service which include:
- Las Vegas, NV
- Reno, NV
- Salt Lake City, UT
- Seattle, WA
- Los Angeles, CA
- San Francisco, CA

The graphs follow the graphs as illustrated in figures above for both the hub-and-spoke and point-to-point model. The costs along each edge were calculated using Google Maps flight estimates between city pairs. To generate our list of flights, I used the population of each city in my list and created a probability array out of the population. The city's with the higher populations were more likely to be included as either a destination or arrival airplane.

The results of the code are discussed in the next section.

# Discussion

## Results

After generating a randomized sample of $100$ flights, the optimization problem as discussed was ran for both graph structures. The resulting optimal values - i.e. total flight hours - for the graph structures are defined in the table below.

| Hub-and-spoke Graph | Point-to-point Graph |
|:-:|:-:|
|3304 flight hours|3035 flight hours|

While the results shown in the table above is the result for a random sample of $100$ flights, the trend of the point-to-point having a lower cost (i.e. total flight hours) than the hub-and-spoke was consistent for numerous samples of flights. Across multiple test samples, the average difference in time ranged from $5 - 10\%$. While this difference may not seem substantial, this is considered a significant amount of savings in the context of the small margin aviation business.

Upon reviewing the results, it seems that the point-to-point graph is more likely to succeed because it provides greater versatility for travel *from* the largest cities (namely Los Angeles). As discussed in previous sections, the randomly sampled flights was weighted with respect to population size of departure cities. In the point-to-point graph, travelers from these larger cities are more likely to immediately travel to their destination city without requiring a connection, given that there is a greater chance to have a direct flight. With the hub-and-spoke model, all flights have to travel through Las Vegas (node 1), which requires a minimum of 1 connecting flight for any flight not including Las Vegas as destination or arrival city.

## Future expansion
- Cost of adding an edge
- Modeling whether it is advantageous to add a city within the network
- Modeling with respect to time
- Account for cost of congestion through specific cities

# References

- https://aeronauticsonline.com/the-airline-economics-of-the-bicycle-wheel-point-to-point-vs-hub-and-spoke-flying/
- https://transportgeography.org/contents/chapter2/geography-of-transportation-networks/point-to-point-versus-hub-and-spoke-network/#:~:text=A%20point%2Dto%2Dpoint%20network,intermediary%20location%20called%20a%20hub
- https://simpleflying.com/how-do-airlines-plan-routes/